In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib


#Step 1: Load the dataset


In [5]:
# Upload dataset from local
from google.colab import files
uploades = files.upload()

df = pd.read_csv("train-dataset.csv")
df.head()

Saving train-dataset.csv to train-dataset.csv


<ipython-input-5-6dd00d695e8a>:5: DtypeWarning: Columns (5,7,8,9,11,12,13,22,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train-dataset.csv")


,ID,Loan Amount,Funded Amount,Funded Amount Investor,Term,Batch Enrolled,Interest Rate,Grade,Sub Grade,Employment Duration,...,Recoveries,Collection Recovery Fee,Collection 12 months Medical,Application Type,Last week Pay,Accounts Delinquent,Total Collection Amount,Total Current Balance,Total Revolving Credit Limit,Loan Status
0,65087372.0,10000.0,32236.0,12329.36286,59.0,BAT2522922,11.135007,B,C4,MORTGAGE,...,2.498291,0.793724,0.0,INDIVIDUAL,49.0,0.0,31.0,311301.0,6619.0,0.0
1,1450153.0,3609.0,11940.0,12191.99692,59.0,BAT1586599,12.237563,C,D3,RENT,...,2.377215,0.974821,0.0,INDIVIDUAL,109.0,0.0,53.0,182610.0,20885.0,0.0
2,1969101.0,28276.0,9311.0,21603.22455,59.0,BAT2136391,12.545884,F,D4,MORTGAGE,...,4.316277,1.020075,0.0,INDIVIDUAL,66.0,0.0,34.0,89801.0,26155.0,0.0
3,6651430.0,11170.0,6954.0,17877.15585,59.0,BAT2428731,16.731201,C,C3,MORTGAGE,...,0.107020,0.749971,0.0,INDIVIDUAL,39.0,0.0,40.0,9189.0,60214.0,0.0
4,14354669.0,16890.0,13226.0,13539.92667,59.0,BAT5341619,15.008300,C,D4,MORTGAGE,...,1294.818751,0.368953,0.0,INDIVIDUAL,18.0,0.0,430.0,126029.0,22579.0,0.0


#Step 2: Drop rows with missing values


In [8]:
df_cleaned = df.dropna()
df_cleaned.shape


(184, 35)

#Step 3: Select useful features and target


In [10]:
selected_features = [
    'Loan Amount', 'Funded Amount', 'Funded Amount Investor', 'Term',
    'Interest Rate', 'Grade', 'Employment Duration', 'Home Ownership',
    'Verification Status', 'Debit to Income', 'Delinquency - two years',
    'Inquires - six months', 'Open Account', 'Public Record',
    'Revolving Balance', 'Revolving Utilities', 'Total Accounts',
    'Application Type', 'Accounts Delinquent', 'Total Current Balance',
    'Total Revolving Credit Limit'
]


X = df_cleaned[selected_features]
y = df_cleaned['Loan Status']

#Step 4: Encode categorical features


In [12]:
# Make a copy to avoid SettingWithCopyWarning
X = X.copy()

# Encode categorical features safely
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X.loc[:, col] = le.fit_transform(X[col])
    label_encoders[col] = le


#Step 5: Train/test split


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#Step 6: Train Logistic Regression model


In [14]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

#Step 7: Evaluate the model


In [15]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97        35
         1.0       0.00      0.00      0.00         2

    accuracy                           0.95        37
   macro avg       0.47      0.50      0.49        37
weighted avg       0.89      0.95      0.92        37



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#Step 8: Save the model and encoders


In [ ]:
joblib.dump(model, "loan_default_model.pkl")
joblib.dump(label_encoders, "label_encoders.pkl")